___Chapter 10: Problem 7___

Fit a neural network to the Default data. Use a single hidden layer with 10 units, and dropout regularization. Have a look at Labs 10.9.1-10.9.2 for guidance. Compare the classification performance of your model with that of linear logistic regression.

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from ISLP.models import ModelSpec as MS
from ISLP.torch import SimpleDataModule, SimpleModule, ErrorTracker
from pytorch_lightning import Trainer

C:\Users\kruti\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load and preprocess data

In [2]:
default = pd.read_csv("Default.csv")
default['default'] = pd.get_dummies(default['default'], drop_first=True).astype(int)
default['student'] = pd.get_dummies(default['student'], drop_first=True).astype(int)

Features, Target and Train-Test Split

In [3]:
model = MS(default.columns.drop('default'), intercept=False)
X = model.fit_transform(default).to_numpy()
Y = default['default'].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=1/3, random_state=1
)

Logistic Regression - Baseline Results

In [4]:
print("LOGISTIC REGRESSION")
print()

def_clf = LogisticRegression(max_iter=1000).fit(X_train, Y_train)
Yhat_class = def_clf.predict(X_test)
lr_accuracy = accuracy_score(Y_test, Yhat_class)

print(f"Test Accuracy = {lr_accuracy:.4f} ({lr_accuracy*100:.2f}%)")

LOGISTIC REGRESSION

Test Accuracy = 0.9763 (97.63%)


Neural Networks - Prepare PyTorch datasets, Create Data Modules, Train and Test

In [5]:
print("NEURAL NETWORK (10 hidden units, 0.4 dropout)")
print()

# Define neural network architecture
class Default(nn.Module):
    def __init__(self, input_size):
        super(Default, self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 10),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(10, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.sequential(x)
    
X_train_t = torch.tensor(X_train.astype(np.float32))
Y_train_t = torch.tensor(Y_train.astype(np.int64))
def_train = TensorDataset(X_train_t, Y_train_t)

X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.int64))
def_test = TensorDataset(X_test_t, Y_test_t)

def_dm = SimpleDataModule(
    def_train,
    def_test,
    batch_size=32,
    num_workers=4,
    validation=def_test
)

def_model = Default(X.shape[1])
def_module = SimpleModule.classification(def_model, num_classes=2)

def_trainer = Trainer(
    max_epochs=50,
    log_every_n_steps=5,
    callbacks=[ErrorTracker()],
    enable_progress_bar=True
)

def_trainer.fit(def_module, datamodule=def_dm)

test_results = def_trainer.test(def_module, datamodule=def_dm)
nn_accuracy = test_results[0]['test_accuracy']
nn_loss = test_results[0]['test_loss']

print(f"\nTest Accuracy: {nn_accuracy:.4f} ({nn_accuracy*100:.2f}%)")
print(f"Test Loss: {nn_loss:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\kruti\AppData\Roaming\Python\Python313\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | Default          | 62     |

NEURAL NETWORK (10 hidden units, 0.4 dropout)

Epoch 49: 100%|██████████| 209/209 [00:01<00:00, 106.99it/s, v_num=1]       

`Trainer.fit` stopped: `max_epochs=50` reached.


Testing DataLoader 0: 100%|██████████| 105/105 [00:00<00:00, 531.10it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.970005989074707     │
│         test_loss         │    0.13501016795635223    │
└───────────────────────────┴───────────────────────────┘


Test Accuracy: 0.9700 (97.00%)
Test Loss: 0.1350


Comparison

In [6]:
print(f"Logistic Regression Accuracy: {lr_accuracy:.4f} ({lr_accuracy*100:.2f}%)")
print(f"Neural Network Accuracy:      {nn_accuracy:.4f} ({nn_accuracy*100:.2f}%)")

print()

print(f"\nDifference: {(lr_accuracy - nn_accuracy)*100:.2f} percentage points")

print()

Logistic Regression Accuracy: 0.9763 (97.63%)
Neural Network Accuracy:      0.9700 (97.00%)


Difference: 0.63 percentage points



Conclusion: The logistic regression model performs slightly better on this dataset, suggesting that the relationship is primarily linear and doesn't require the added complexity of a neural network.